In [141]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib

from pandas import DataFrame
from pandas import read_csv

팀 리스트 가져오기

In [142]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Referer': 'https://www.koreabaseball.com/Player/Search.aspx',
    'Origin': 'https://www.koreabaseball.com',
    'Content-Type': 'application/x-www-form-urlencoded',
}

In [143]:
rq = requests.Session()

In [144]:
src = BeautifulSoup(rq.get('https://www.koreabaseball.com/Player/Search.aspx', headers=headers).text)
teams = [(team.text, team['value']) for team in src.find(class_='select02').find_all('option')][1:]

In [145]:

for x,y in teams:
    print x, y

NC NC
두산 OB
KT KT
LG LG
키움 WO
KIA HT
롯데 LT
삼성 SS
SK SK
한화 HH


팀별 선수목록 가져오기

In [146]:
def make_post_next_pageset(code):
    return urllib.urlencode({        
        '__EVENTTARGET':'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ucPager$btnLast',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        '__VIEWSTATE': __VIEWSTATE,
        '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
        '__EVENTVALIDATION': __EVENTVALIDATION,
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$udpRecord': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$hfPage': __PREVPAGE,
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam': str(code),
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlPosition': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$txtSearchPlayerName': ''
           })

In [147]:
def make_post_page(code, page):
    return urllib.urlencode({
        '__EVENTTARGET':'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ucPager$btnNo' + str(page % 5 + 5*(page % 5 == 0)),
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        '__VIEWSTATE': __VIEWSTATE,          
        '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
        '__EVENTVALIDATION': __EVENTVALIDATION,
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$udpRecord': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$hfPage': __PREVPAGE,
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam': str(code),
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlPosition': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$txtSearchPlayerName': ''
           })

In [148]:
def make_post_data(code, page=1):
    return urllib.urlencode({
        '__EVENTTARGET':'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': '',
        '__VIEWSTATE': __VIEWSTATE,
        '__VIEWSTATEGENERATOR': __VIEWSTATEGENERATOR,
        '__EVENTVALIDATION': __EVENTVALIDATION,        
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$udpRecord': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$hfPage': '1',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam': str(code),
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlPosition': '',
        'ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$txtSearchPlayerName': ''
           })

In [149]:
def update_state(res, page=1):
    global __VIEWSTATE, __EVENTVALIDATION, __VIEWSTATEGENERATOR, __PREVPAGE
    src = BeautifulSoup(res)
    if not src.find('input', {'name':'__VIEWSTATE'}):
        # Exception!!
        # print src
        pass
    __PREVPAGE = str(page)
    __VIEWSTATE = src.find('input', {'name':'__VIEWSTATE'})['value']
    __VIEWSTATEGENERATOR = src.find(id='__VIEWSTATEGENERATOR')['value']
    __EVENTVALIDATION = src.find(id='__EVENTVALIDATION')['value']

In [150]:
def get_player_list(code):
    rq = requests.Session()
    res = rq.get('https://www.koreabaseball.com/Player/Search.aspx', headers=headers)
    update_state(res.text)
    
    batters = []; pitchers = []
    
    for i in xrange(1,9):
        if i % 5 == 1:
            if i > 1:
                res = rq.post('https://www.koreabaseball.com/Player/Search.aspx', data=make_post_next_pageset(code), headers=headers)
            else:
                res = rq.post('https://www.koreabaseball.com/Player/Search.aspx', data=make_post_data(code, i), headers=headers)
            try:
                update_state(res.text)
            except:
                print 'Break at %d' % i
                break
        res = rq.post('https://www.koreabaseball.com/Player/Search.aspx', data=make_post_page(code, i), headers=headers)
        try:
            update_state(res.text, i)
        except:
            print 'Break at %d' % i
            break
        src = BeautifulSoup(res.text)
        if src.find(class_='tEx') == None:
            print code, i, make_post_page(code, i), res.text
            break
        rows = src.find(class_='tEx').find('tbody').find_all('tr')
        print '%s_%d: %d' % (code, i, len(rows))
        if len(rows) == 0:
            break
        for j in xrange(len(rows)):
            link = rows[j].find('a')
            if link['href'].find('HitterDetail') != -1:
                batters.append(link['href'].split('playerId=')[-1])
            if link['href'].find('PitcherDetail') != -1:
                pitchers.append(link['href'].split('playerId=')[-1])
    return (batters, pitchers)
    

In [151]:
player_list = {
               'SS': None,
               'WO': None,
               'NC': None,
               'OB': None,
               'LG': None,
               'SK': None,
               'KT': None,
               'LT': None,
               'HT': None,
               'HH': None
              }

In [152]:
for team in player_list.keys():
    player_list[team] = get_player_list(team)

SK_1: 20
SK_2: 20
SK_3: 20
SK_4: 20
SK_5: 11
SK_6: 20
SK_7: 20
SK_8: 20
WO_1: 20
WO_2: 20
WO_3: 20
WO_4: 20
WO_5: 1
WO_6: 20
WO_7: 20
WO_8: 20
LT_1: 20
LT_2: 20
LT_3: 20
LT_4: 20
LT_5: 6
LT_6: 20
LT_7: 20
LT_8: 20
KT_1: 20
KT_2: 20
KT_3: 20
KT_4: 20
KT_5: 17
KT_6: 20
KT_7: 20
KT_8: 20
HH_1: 20
HH_2: 20
HH_3: 20
HH_4: 20
HH_5: 16
HH_6: 20
HH_7: 20
HH_8: 20
LG_1: 20
LG_2: 20
LG_3: 20
LG_4: 20
LG_5: 5
LG_6: 20
LG_7: 20
LG_8: 20
SS_1: 20
SS_2: 20
SS_3: 20
SS_4: 20
SS_5: 4
SS_6: 20
SS_7: 20
SS_8: 20
NC_1: 20
NC_2: 20
NC_3: 20
NC_4: 20
NC_5: 14
NC_6: 20
NC_7: 20
NC_8: 20
OB_1: 20
OB_2: 20
OB_3: 20
OB_4: 20
OB_5: 16
OB_6: 20
OB_7: 20
OB_8: 20
HT_1: 20
HT_2: 20
HT_3: 20
HT_4: 20
HT_5: 8
HT_6: 20
HT_7: 20
HT_8: 20


선수별 데이터 가져오기

In [14]:
def get_basic_info(src):
    items = src.find(class_='player_basic').find_all('li')
    dic = {}
    
    for item in items:
        dic[item.find('strong').text.replace(':', '')] = item.find('span').text
    
    return dic

투수 정보 가져오기

In [15]:
def get_pitcher_info(id):
    url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId=' + str(id)
    src = BeautifulSoup(rq.get(url, headers=headers).text)
    
    tables = src.find_all(class_='tt')[:2]
    ret = {}
    
    for table in tables:
        keys = [x.find('a')['title'].strip() if x and x.find('a') else x.text for x in table.find('thead').find_all('th')]
        values = [x.text for x in table.find('tbody').find_all('td')]
        
        if len(values) != len(keys) or values.count('-') > 0:
            return None
        
        fvalues = []

        def frac_to_float(val):
            return float(val.split('/')[0]) / float(val.split('/')[-1])

        for key, val in zip(keys, values):
            if key == u'팀명':
                fvalues.append(val)
                continue
                
            try:
                fvalues.append(float(val))
            except:
                if val.find(' ') != -1:
                    fvalues.append(sum([frac_to_float(x) if x.find('/') != -1 else float(x) for x in val.split()]))
                    
        ret.update(dict(zip(keys, fvalues)))
        
    ret.update(get_basic_info(src))
    return ret

타자정보가져오기

In [16]:
def get_batter_info(id):
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=' + str(id)
    src = BeautifulSoup(rq.get(url, headers=headers).text)
    
    tables = src.find_all(class_='tt')[:2]
    ret = {}
    
    for table in tables:
        keys = [x.find('a')['title'].strip() if x and x.find('a') else x.text for x in table.find('thead').find_all('th')]
        values = [x.text for x in table.find('tbody').find_all('td')]
        
        if len(values) != len(keys) or values.count('-') > 0:
            return None
        
        fvalues = []

        def frac_to_float(val):
            return float(val.split('/')[0]) / float(val.split('/')[-1])

        for key, val in zip(keys, values):
            if key == u'팀명':
                fvalues.append(val)
                continue
                
            try:
                fvalues.append(float(val))
            except:
                if val.find(' ') != -1:
                    fvalues.append(sum([frac_to_float(x) if x.find('/') != -1 else float(x) for x in val.split()]))
                    
        ret.update(dict(zip(keys, fvalues)))
        
    ret.update(get_basic_info(src))
    return ret

CSV 저장하기

In [17]:
from collections import defaultdict

def merge_dicts(dicts):
    ret = defaultdict(list)
    
    for d in dicts:
        for k in d.keys():
            ret[k].append(d[k])
    
    return ret

In [18]:
def get_batter_info(id):
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=' + str(id)
    src = BeautifulSoup(rq.get(url, headers=headers).text)
    
    tables = src.find_all(class_='tt')[:2]
    ret = {}
    
    for table in tables:
        keys = [x.find('a')['title'].strip() if x and x.find('a') else x.text for x in table.find('thead').find_all('th')]
        values = [x.text for x in table.find('tbody').find_all('td')]
        
        if len(values) != len(keys) or values.count('-') > 0:
            return None
        
        fvalues = []

        def frac_to_float(val):
            return float(val.split('/')[0]) / float(val.split('/')[-1])

        for key, val in zip(keys, values):
            if key == u'팀명':
                fvalues.append(val)
                continue
                
            try:
                fvalues.append(float(val))
            except:
                if val.find(' ') != -1:
                    fvalues.append(sum([frac_to_float(x) if x.find('/') != -1 else float(x) for x in val.split()]))
                    
        ret.update(dict(zip(keys, fvalues)))
        
    ret.update(get_basic_info(src))
    return ret

csv 저장

In [19]:
from collections import defaultdict

def merge_dicts(dicts):
    ret = defaultdict(list)
    
    for d in dicts:
        for k in d.keys():
            ret[k].append(d[k])
    
    return ret

In [20]:
def get_players_in_team(code):
    ret = {
        'batters': [],
        'pitchers': []
    }
    
    for men in player_list[code][0]:
        r = get_batter_info(men)
        if r:
            ret['batters'].append(r)
    for men in player_list[code][1]:
        r = get_pitcher_info(men)
        if r:
            ret['pitchers'].append(r)
            
    ret['batters'] = merge_dicts(ret['batters'])
    ret['pitchers'] = merge_dicts(ret['pitchers'])
    
    return ret

In [21]:
SS = get_players_in_team('SS')
SSBatter = pd.DataFrame(SS['batters'])
SSPitcher = pd.DataFrame(SS['pitchers'])

In [22]:
NC = get_players_in_team('NC')
NCBatter = pd.DataFrame(NC['batters'])
NCPitcher = pd.DataFrame(NC['pitchers'])

In [24]:
#WO = get_players_in_team('WO')
#WOBatter = pd.DataFrame(WO['batters'])
#WOPitcher = pd.DataFrame(WO['pitchers'])

In [25]:
OB = get_players_in_team('OB')
OBBatter = pd.DataFrame(OB['batters'])
OBPitcher = pd.DataFrame(OB['pitchers'])

In [26]:

SK = get_players_in_team('SK')
SKBatter = pd.DataFrame  (SK['batters'])
SKPitcher = pd.DataFrame (SK['pitchers'])

In [27]:
LG = get_players_in_team('LG')
LGBatter = pd.DataFrame  (LG['batters'])
LGPitcher = pd.DataFrame (LG['pitchers'])

In [28]:
LT = get_players_in_team('LT')
LTBatter = pd.DataFrame  (LT['batters'])
LTPitcher = pd.DataFrame (LT['pitchers'])

In [29]:
HT = get_players_in_team('HT')
HTBatter = pd.DataFrame  (HT['batters'])
HTPitcher = pd.DataFrame (HT['pitchers'])

In [30]:
HH = get_players_in_team('HH')
HHBatter = pd.DataFrame  (HH['batters'])
HHPitcher = pd.DataFrame (HH['pitchers'])

In [31]:
KT = get_players_in_team('KT')
KTBatter = pd.DataFrame  (KT['batters'])
KTPitcher = pd.DataFrame (KT['pitchers'])

In [32]:

LGBatter.to_csv('data/lg_batters.csv', encoding='utf-8', index=False)
LGPitcher.to_csv('data/lg_pitchers.csv', encoding='utf-8', index=False)

SKBatter.to_csv('data/sk_batters.csv', encoding='utf-8', index=False)
SKPitcher.to_csv('data/sk_pitchers.csv', encoding='utf-8', index=False)

LTBatter.to_csv('data/lt_batters.csv', encoding='utf-8', index=False)
LTPitcher.to_csv('data/lt_pitchers.csv', encoding='utf-8', index=False)

HTBatter.to_csv('data/kia_batters.csv', encoding='utf-8', index=False)
HTPitcher.to_csv('data/kia_pitchers.csv', encoding='utf-8', index=False)

HHBatter.to_csv('data/hh_batters.csv', encoding='utf-8', index=False)
HHPitcher.to_csv('data/hh_pitchers.csv', encoding='utf-8', index=False)

In [33]:
LGBatter.to_csv( 'data/lg_batters_windows.csv',   encoding='euc_kr', index=False)
LGPitcher.to_csv('data/lg_pitchers_windows.csv',  encoding='euc_kr', index=False)

SKBatter.to_csv( 'data/sk_batters_windows.csv',   encoding='euc_kr', index=False)
SKPitcher.to_csv('data/sk_pitchers_windows.csv',  encoding='euc_kr', index=False)

LTBatter.to_csv( 'data/lt_batters_windows.csv',   encoding='euc_kr', index=False)
LTPitcher.to_csv('data/lt_pitchers_windows.csv',  encoding='euc_kr', index=False)

HTBatter.to_csv( 'data/kia_batters_windows.csv',   encoding='euc_kr', index=False)
HTPitcher.to_csv('data/kia_pitchers_windows.csv',  encoding='euc_kr', index=False)

HHBatter.to_csv( 'data/hh_batters_windows.csv',   encoding='euc_kr', index=False)
HHPitcher.to_csv('data/hh_pitchers_windows.csv',  encoding='euc_kr', index=False)

In [35]:
SSBatter.to_csv('data/ss_batters.csv', encoding='utf-8', index=False)
SSPitcher.to_csv('data/ss_pitchers.csv', encoding='utf-8', index=False)

NCBatter.to_csv('data/nc_batters.csv', encoding='utf-8', index=False)
NCPitcher.to_csv('data/nc_pitchers.csv', encoding='utf-8', index=False)

#WOBatter.to_csv('data/nx_batters.csv', encoding='utf-8', index=False)
#WOPitcher.to_csv('data/nx_pitchers.csv', encoding='utf-8', index=False)

OBBatter.to_csv('data/ds_batters.csv', encoding='utf-8', index=False)
OBPitcher.to_csv('data/ds_pitchers.csv', encoding='utf-8', index=False)

KTBatter.to_csv('data/kt_batters.csv', encoding='utf-8', index=False)
KTPitcher.to_csv('data/kt_pitchers.csv', encoding='utf-8', index=False)


In [37]:
SSBatter.to_csv('data/ss_batters_windows.csv', encoding='euc-kr', index=False)
SSPitcher.to_csv('data/ss_pitchers_windows.csv', encoding='euc-kr', index=False)

NCBatter.to_csv('data/nc_batters_windows.csv', encoding='euc-kr', index=False)
NCPitcher.to_csv('data/nc_pitchers_windows.csv', encoding='euc-kr', index=False)

#WOBatter.to_csv('data/nx_batters_windows.csv', encoding='euc-kr', index=False)
#WOPitcher.to_csv('data/nx_pitchers_windows.csv', encoding='euc-kr', index=False)

OBBatter.to_csv('data/ds_batters_windows.csv', encoding='euc-kr', index=False)
OBPitcher.to_csv('data/ds_pitchers_windows.csv', encoding='euc-kr', index=False)

KTBatter.to_csv('data/kt_batters_windows.csv', encoding='euc-kr', index=False)
KTPitcher.to_csv('data/kt_pitchers_windows.csv', encoding='euc-kr', index=False)



팀별 승무패 가져오기

In [68]:

soup = BeautifulSoup(requests.get('https://www.koreabaseball.com/TeamRank/TeamRank.aspx', headers=headers).text)

In [117]:
WDLtable = soup.find_all('table', class_='tData')[-1]
WDLhdr = WDLtable.find('thead').find_all('th')
WDLrows = WDLtable.find('tbody').find_all('tr')

In [173]:

teamseq = [x.text.split('(')[0] for x in WDLhdr][1:-1]

In [174]:
print " ".join(teamseq)

NC 두산 KT LG 키움 KIA 롯데 삼성 SK 한화


In [171]:
teams = ['ss', 'nx', 'nc', 'lg', 'sk', 'ds', 'lt', 'kia', 'hh','kt']

In [207]:
from copy import deepcopy

WDLtable = {team1: {team2: deepcopy({}) for team2 in teams if team1 != team2} for team1 in teams}

i = 0; j = 0
for row in WDLrows[:-1]:
    j = 0
    for col in row.find_all('td')[1:len(teamseq)]:
        if i != j:
            s = col.text.strip().split('-')
            WDLtable[teams[i]][teams[j]]['W'] = int(s[0])
            WDLtable[teams[i]][teams[j]]['L'] = int(s[1])
            WDLtable[teams[i]][teams[j]]['D'] = int(s[2])
        j += 1
    i += 1  

WDLtable['ds']['ss'] 이렇게 하면 두산의 삼성에 대한 승패무를 볼 수 있음(W/L/D):

WDLtable['ds']['ss']['W'] : 승

WDLtable['ds']['ss']['L'] : 패

WDLtable['ds']['ss']['D'] : 무

In [208]:
import json
with open('data/WDL1.json', 'w') as fout:
    print >> fout, json.dumps(WDLtable)

In [209]:
WDL = pd.DataFrame(WDLtable)


In [210]:
WDL.to_csv('data/WDL6.csv', encoding='utf-8', index=False)

In [44]:
import json
with open('data/WDL.csv', 'w') as fout:
    print >> fout, json.dumps(WDLtable)